##***creating catalog***



In [0]:
%sql

create catalog ansh_lamba;


In [0]:
%sql
-- Set the catalog context
USE CATALOG ansh_lamba;

-- Create schema under the specified catalog
CREATE SCHEMA salesdata;

In [0]:
%sql
show schemas;

In [0]:
%sql
--  creating managed table

create table salesdata.sales_data (id int, name varchar(10))

## Triggering email when scema changes
### Using Logic APP

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Reference schema (stored schema)
reference_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

# Load new data
df = spark.table('ansh_lamba.salesdata.silver')

new_schema = df.schema

def schemas_are_equal(schema1, schema2):
    return schema1.simpleString() == schema2.simpleString()

# Detect schema changes
if not schemas_are_equal(reference_schema, new_schema):
    print("Schema mismatch detected!")

    # Logic App trigger using Webhook URL
    import requests
    import json

    # Webhook URL of Logic App
    webhook_url = "https://prod-07.southindia.logic.azure.com:443/workflows/5b91112106bc4340855f9201705342f2/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=sRusFLDIpAZ87XQnupQ76PDLALE1e4cyBFCrVy0cxAA"

    # Example payload to send to Logic App
    payload = {
    "schema_info": {
        "old_schema": reference_schema.simpleString(),
        "new_schema": new_schema.simpleString()
    },
    "message": "Schema has changed! Please review. cjbsjvhdsjvhsdmvndsmvndvnsmv zvndzbv mdzv bmzdbv ",
    "to": "pavangowda1706@outlook.com"
    }

    response = requests.post(webhook_url, headers={'Content-Type': 'application/json'}, data=json.dumps(payload))

    if response.status_code == 200 | 202 :
        print("Notification sent successfully!")
    else:
        print(f"Failed to send notification. Status code: {response.status_code}")

else:
    print("Schema is up to date!")


## **Handling schema evaluation**

**NOTE** - **In delta table we can handled by merge schema**



In [0]:
from pyspark.sql.functions import lit

def adjust_schema(dataframe, reference_schema):
    for field in reference_schema.fields:
        if field.name not in dataframe.columns:
            dataframe = dataframe.withColumn(field.name, lit(None).cast(field.dataType))
        else:
            pass
    
    # Remove extra columns and reorder
    dataframe = dataframe.select([field.name for field in reference_schema.fields if field.name in dataframe.columns])
    return dataframe

# Adjust schema of new_data
new_data = adjust_schema(df, reference_schema)

## we can derive the dataframe baased on need
display(new_data)
